#function

In [18]:
from ortools.graph import pywrapgraph
import os
import numpy as np
import subprocess
import copy
import operator
from graphviz import Digraph
from IPython.display import IFrame
from nltk.stem.porter import *
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
#tg = 'zinc finger'
def query_path(tg,d):
    #tg = 'zmym2'
    #d='lung adenocarcinoma'
    pre = {}
    visited = set()
    visited.add(tg)
    net = ect
    find = False
    all_node = set()
    all_node.add(tg)
    for iter in range(100):
        new_node = set()
        for ss in visited:
            if ss == d:
                tmp = ss
                while tmp in pre:
                    print tmp,node_type[tmp],'->',pre[tmp]
                    tmp = pre[tmp]
                find = True
                break
            if ss not in net:
                continue
            for e in net[ss]:
                if e in all_node:
                    continue
                new_node.add(e) 
                all_node.add(e)
                pre[e] = ss
        visited = new_node     
        if find:
            break
            
def plot_figure(node_list, edge_list,output_file,net_edge_weight):
    dot = Digraph(comment='Network flow')
    for node in node_list:
        if node==source_node:
            dot.attr('node',color='blue')
            dot.node(node)
        elif node in d2g[d]:
            dot.attr('node',color='red')
            dot.node(node) 
        elif node in node_type and node_type[node] == 'function':
            dot.attr('node',color='green')
            dot.node(node)          
        elif node in node_type and node_type[node] == 'symptom':
            dot.attr('node',color='yellow')
            dot.node(node)  
        else:        
            dot.attr('node',color='black')
            dot.node(node)   
    for e in edge_list:        
        e1,e2,w,mw = e
        if e1 in kg_relation and e2 in kg_relation[e1]:
            label = 'Percha et al.'
        else:
            label = 'text mining'
        if e1 not in node_type:
            t1 = 'entity'
        else:
            t1 = node_type[e1]
        if e2 not in node_type:
            t2 = 'entity'
        else:
            t2 = node_type[e2]
        if e1 not in wct:
            wct[e1] = 0
        if e2 not in wct:
            wct[e2] = 0
        edge_weight = 0
        if e1 in net_edge_weight and e2 in net_edge_weight[e1]:
            edge_weight = net_edge_weight[e1][e2]
        dot.edge(e1,e2,label=label+'('+t1+'-'+t2+':'+str(w)+')'+str(edge_weight))
    dot.render(output_file) 
    
def dfs(cur_node,target_node,net_flow_edge,node_type,edge_type,max_depth,cur_depth,path,global_all_path):
    #print cur_node,target_node,cur_depth,max_depth,net_flow_edge[cur_node]
    if cur_node == target_node:
        #print path
        global_all_path.append(path)
        return global_all_path
    if cur_depth == max_depth:
        #print path
        return []
    #print cur_depth
    argu_path = []
    for ngh in net_flow_edge[cur_node]:
        if True:#[node_type[cur_node],node_type[ngh]] in edge_type:
            #print ngh,path
            new_path = dfs(ngh,target_node,net_flow_edge,node_type,edge_type,max_depth,cur_depth+1,path+'\t'+ngh,global_all_path)
            #if len(new_path) > 0:
            #    argu_path.append(new_path)
    return global_all_path

def dfs_find_graph(edge_type,source_node,target_node,select_node_set,net_flow_edge,node2i,i2node,node_type,max_depth = 5):    
    visited = set()
    all_path = dfs(source_node,target_node,net_flow_edge,node_type,edge_type,max_depth,0,source_node,[])
    npath = len(all_path)
    edge_d = {}
    node_list = set()
    #print npath
    #print all_path
    for p in range(npath):
        path = all_path[p].split('\t')
        for w in range(0,len(path)-1):
            node_list.add(path[w])
            node_list.add(path[w+1])
            edge_d[(path[w],path[w+1])] = edge_d.get((path[w],path[w+1]),0) + 1
    extracted_net_flow_edge = {}
    edge_list = []
    extracted_select_node_set = set()
    for k in edge_d:
        edge_list.append((k[0],k[1],edge_d[k],100))
        if k[0] not in extracted_net_flow_edge:
            extracted_net_flow_edge[k[0]] = {}
        extracted_net_flow_edge[k[0]][k[1]] = net_flow_edge[k[0]][k[1]]
        extracted_select_node_set.add(k[0])
        extracted_select_node_set.add(k[1])
    extracted_select_node_set.add('@super target')    
    extracted_net_flow_edge['@super target'] = {}
    extracted_net_flow_edge[target_node]['@super target'] = 1000000
    extracted_net_flow_edge['@super target'][target_node] = 1000000
        
    edge_list, node_list = run_network_flow('tmp',source_node,extracted_select_node_set,extracted_net_flow_edge,node2i,i2node)
    return edge_list,node_list

#print source_node,g
#edge_list, node_list = dfs_find_graph(edge_type,source_node,g,select_node_set,net_flow_edge,node2i,i2node,node_type,6)

def run_network_flow(fout_file,source_node,select_node_set,net_flow_edge,node2i,i2node):
    edge_list = []
    node_list = set()
    nedge = -1
    for edge_mult in [100]:
        max_flow = pywrapgraph.SimpleMaxFlow()    
        for g1 in select_node_set:
            for g2 in net_flow_edge[g1]:
                #if g2 not in select_node_set:
                #    continue
                #print node2i[g1],node2i[g2],net_flow_edge[g1][g2]
                max_flow.AddArcWithCapacity(node2i[g1],node2i[g2],int(net_flow_edge[g1][g2]*edge_mult))
                #print node2i[g1],node2i[g2],int(net_flow_edge[g1][g2]*edge_mult)
                
        if max_flow.Solve(node2i[source_node],node2i['@super target']) == max_flow.OPTIMAL:
            fout = open(out_fname,'w')
            fout.write('source\ttarget\tweight\n')
            #print('Max flow:', max_flow.OptimalFlow())
            #print('')
            #print('  Arc    Flow / Capacity')
            nedge = 0
            for i in range(max_flow.NumArcs()):
                if max_flow.Flow(i)>0:
                    nedge += 1
                    fout.write(i2node[max_flow.Tail(i)]+'\t'+i2node[max_flow.Head(i)]+'\t'+str(max_flow.Flow(i))+'\n')
            fout.close()
        else:
            print('There was an issue with the max flow input.')
        if nedge>1:
            #print('Max flow:', max_flow.OptimalFlow())
            #print('')
            #print('  Arc    Flow / Capacity')
            
            
            for i in range(max_flow.NumArcs()):
                if max_flow.Flow(i)>0:
                    edge_list.append([i2node[max_flow.Tail(i)],i2node[max_flow.Head(i)],max_flow.Flow(i),max_flow.Capacity(i)])
                    node_list.add(i2node[max_flow.Tail(i)])
                    node_list.add(i2node[max_flow.Head(i)])
                    #print('%1s -> %1s   %3s  / %3s' % (i2node[max_flow.Tail(i)],i2node[max_flow.Head(i)],max_flow.Flow(i),max_flow.Capacity(i)))
                    pass
            break
    return edge_list,node_list

def extract_sub_graph(select_edge_ct,rev_select_edge_ct,source_node,target_set,max_path_L = 3, edge_thres = 0.1 ,max_ngh = 50):    
    global dis2target,dis2source
    
    bp_netflow = collections.defaultdict(dict)
    netflow = collections.defaultdict(dict)
    get_K_source_ngh(source_node,select_edge_ct,max_path_L,edge_thres)
                    
    seed = set()
    seed = seed.union(target_set)    
    for s in seed:
        get_K_target_ngh(s,rev_select_edge_ct,max_path_L,edge_thres,select_edge_ct)
            
    select_node_set = set()
    for s in dis2source[source_node]:
        dis2s = dis2source[source_node].get(s,100)
        dis2t = 100
        for t in target_set:
            dis2t = min(dis2t,dis2target[t].get(s,100))
        total_dis = dis2s + dis2t
        if total_dis > max_path_L:
            continue
        select_node_set.add(s)
        
    select_node_set = select_node_set.union(seed)
    select_node_set.add('@super target')
    for t1 in select_node_set:
        new_d = {}
        for t2 in rev_select_edge_ct[t1]:            
            if t2 in select_node_set:
                #if t1==source_node:
                #    print t1,t2,select_edge_ct[t1][t2],edge_thres
                if rev_select_edge_ct[t1][t2] > edge_thres:
                    new_d[t2] = rev_select_edge_ct[t1][t2] 
        sort_x = sorted(new_d.items(),key=operator.itemgetter(1))   
        sort_x.reverse()
        for i in range(min(len(sort_x),max_ngh)):
            t2 = sort_x[i][0]
            sc = sort_x[i][1]
            bp_netflow[t1][t2] = sc
            netflow[t2][t1] = sc
        
    for t in target_set:
        netflow[t]['@super target'] = 1000000
        netflow['@super target'][t] = 1000000

    node2i = {}
    i2node = {}
    for s in select_node_set:
        nnode = len(node2i)
        i2node[nnode] = s
        node2i[s] = nnode
    
    return netflow,node2i,i2node,select_node_set

def get_auc_per_drug(gene_vec,d2g,d):
    gene_score = gene_vec[d]
    y = []
    pred = []
    #d = 'bladder'
    for g in gene_score:
        if g in d2g[d]:
            y.append(1)
        else:
            y.append(0)
        pred.append(gene_score[g])
    y = np.array(y)
    pred = np.array(pred)
    if len(np.unique(y))<=1:
        return float('nan')
    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    return auc

def get_auc_per_gene(gene_vec,d2g,g):
    y = []
    pred = []
    for d in gene_vec:
        if g in gene_vec[d] and d in d2g:
            if g in d2g[d]:
                y.append(1)
            else:
                y.append(0)
        pred.append(gene_vec[d][g])
    y = np.array(y)
    pred = np.array(pred)
    if len(np.unique(y))<=1:
        return float('nan')
    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    return auc

def get_all_sentences(w1,w2):
    nchunk = 400
    sent_collection = []
    os.chdir('/srv/local/work/swang141/PatientSetAnnotation/Sheng/src/network_flow/')
    for pid in range(nchunk):
        fin = open('../../data/pubmed/pmid2meta_autophrase.chunk'+str(pid))
        ct = 0
        for line in fin:
            text = line.lower().translate(None,',?!:()=%>/[]').strip().strip('.').split('. ')
            sent_l = text
            for sent in sent_l:
                if w1 in sent and w2 in sent:
                    sent_collection.append(sent.replace('.','')+' .')
        fin.close()
        if pid%50==0:
            #print 'read',pid,len(sent_collection)
            pass
    return sent_collection

def get_path(edge,st,ed):
    lmtzr = WordNetLemmatizer()
    stemmer = PorterStemmer()    

    nedge = len(edge)
    net = {}
    for e in edge:
        tp = e.split('(')[0]
        e1 = e.split('(')[1].split(',')[0].lstrip(' ')
        e2 = e.split('(')[1].split(',')[1].split(')')[0].lstrip(' ')
        if st == e2.split('-')[0]:
            st = e2
        if ed == e2.split('-')[0]:
            ed = e2
        #e1 = e1.split('-')[0].lstrip(' ')
        #e2 = e2.split('-')[0].lstrip(' ')
        if e2 not in net:
            net[e2] = {}
        net[e2][e1] = tp
    if st not in net or ed not in net:
        return 'wrong',[]
    cur = ed
    ed_path = [ed.split('-')[0]]
    while cur!='ROOT-0':
        next = net[cur].keys()[0]
        ed_path.append(next.split('-')[0])
        cur = next  
    cur = st
    st_path = [st.split('-')[0]]
    while cur!='ROOT-0':
        next = net[cur].keys()[0]       
        st_path.append(next.split('-')[0])
        cur = next    
    find = False
    for si in range(0,len(st_path)):
        for ei in range(0,len(ed_path)):
            if st_path[si]==ed_path[ei]:
                find = True
                break
        if find:
            break
    path = ''
    word_list = []
    for i in range(0,si):
        path += st_path[i] + ' '
        if st_path[i] != st.split('-')[0]:
            word_list.append(lmtzr.lemmatize(stemmer.stem(st_path[i].decode("utf8"))))
    for i in range(ei,-1,-1):
        path += ed_path[i] + ' '
        if ed_path[i]!=ed.split('-')[0]:
            word_list.append(lmtzr.lemmatize(stemmer.stem(ed_path[i].decode("utf8"))))
    return path, word_list

def find_sent(w1,w2,topk_word = 2):
#w1 = 'hyperphosphatemia'
#w2 = 'calcium acetate'
    sent_l = get_all_sentences(w1,w2)
    os.chdir('/srv/local/work/swang141/PatientSetAnnotation/Sheng/analysis/stanford_parser_java/stanford-parser-full-2017-06-09/')
    new_w2 = w2.replace(' ','_')
    new_w1 = w1.replace(' ','_')
    input_path = 'input.txt'
    #fin = open(input_path)
    #sent_l = []
    #for line in fin:
    #    sent_l.append(line.strip())
    #fin.close()
    fout = open(input_path,'w')
    for s in sent_l:
        fout.write(s.rstrip().lstrip().replace(w2,new_w2).replace(w1,new_w1)+'\n')
    fout.close()
    
    output_path = 'output.txt'
    command = 'java -Xmx2g -cp "*" edu.stanford.nlp.parser.nndep.DependencyParser -model edu/stanford/nlp/models/parser/nndep/english_UD.gz -textFile '+input_path+' -outFile '+output_path
    output = subprocess.check_output(command, shell=True)

    fin = open(output_path)
    edge = []
    path_l = []
    vocab_dct = {}
    stem_word_list_l = []
    for line in fin:
        text = line.strip()
        if text == '':
            path,stem_word_list = get_path(edge,new_w2,new_w1)
            stem_word_list_l.append(stem_word_list)
            path_l.append(path)
            for w in stem_word_list:
                vocab_dct[w] = vocab_dct.get(w,0) + 1
            #print path
            #print '----------------'
            edge = []
        else:
            edge.append(text)
    fin.close()
    os.chdir('/srv/local/work/swang141/PatientSetAnnotation/Sheng/src/network_flow/')
    vocab_list = sorted(vocab_dct.items(),key=operator.itemgetter(1))
    vocab_list.reverse()
    ct = 0
    vocab_score = {}
    for i in vocab_list:
        ct = ct + 1
        if ct > topk_word:
            break
        vocab_score[i[0]] = i[1]
        print i[0],i[1]

    nsent = len(sent_l)
    sent_score = {}
    best_sc = -1
    best_sent = ''
    best_path = ''
    #print nsent,len(path_l),len(sent_l),len(edge)
    assert(nsent == len(path_l))
    for i in range(nsent):
        npath_word = len(path_l[i].split(' '))
        nsent_word = len(sent_l[i].split(' '))
        word_norm = npath_word * 50 + nsent_word
        sc = 0.
        #for ww in path_l[i].split(' '):
        #    if ww not in sent_l[i]:
        #        print sent_l[i],path_l[i]
        for w in stem_word_list_l[i]:
            sc = sc + vocab_score.get(w,0)
        sc = sc / word_norm
        sent_score[sent_l[i]] = sc
        if sc > best_sc:            
            best_sc = sc
            best_sent = sent_l[i]
            best_path = path_l[i]
    #sent_list = sorted(sent_score.items(),key=operator.itemgetter(1))
    #sent_list.reverse()    
    #for i in range(0,3):
    #    print sent_list[i][0],sent_list[i][1]
    return best_sent,best_path,best_sc,path_l


def extract_sent(ect_ref,e1,e2):
    if e1 in kg_relation and e2 in kg_relation[e1]:
        sent = kg_relation[e1][e2][0]
        return sent
    pid,cid,sid,d = ect_ref[e1][e2].split('\t')[0].split(',')
    sid = int(sid)
    cid = int(cid)
    fin = open('../../data/pubmed/pmid2meta_autophrase.chunk'+pid)
    ct = 0
    for line in fin:
        ct = ct + 1
        if ct==int(cid):
            sl = line.strip().split('.')
            sent = sl[sid]
            break
    fin.close()
    return sent

def get_K_source_ngh(s,net,max_path_L,edge_thres): 
    global dis2source
    if s in dis2source:
        return 
    dis2source[s] = {}
    visited = set()
    visited.add(s)
    dis2source[s][s] = 0
    for iter in range(max_path_L):
        new_node = set()
        for ss in visited:
            if ss not in net:
                continue
            for e in net[ss]:
                if e in dis2source[s] or net[ss][e] < edge_thres:
                    continue
                new_node.add(e) 
        visited = new_node
        for ss in visited:
            if ss not in dis2source[s]:
                dis2source[s][ss] = iter    
    return
                    
def get_K_target_ngh(s,rev_net,max_path_L,edge_thres,net): 
    global dis2target
    if s in dis2target:
        return 
    dis2target[s] = {}
    visited = set()
    visited.add(s)
    dis2target[s][s] = 0
    for iter in range(max_path_L):
        new_node = set()
        for ss in visited:
            if ss not in rev_net:
                continue
            for e in rev_net[ss]:
                if e in dis2target[s]  or net[e][ss] < edge_thres:
                    continue
                new_node.add(e) 
        visited = new_node
        for ss in visited:
            if ss not in dis2target[s]:
                dis2target[s][ss] = iter+1
        
    return

def run_network_flow(fout_file,source_node,select_node_set,net_flow_edge,node2i,i2node):
    edge_list = []
    node_list = set()
    nedge = -1
    for edge_mult in [10]:
        max_flow = pywrapgraph.SimpleMaxFlow()    
        for g1 in select_node_set:
            for g2 in net_flow_edge[g1]:
                #if g2 not in select_node_set:
                #    continue
                #print node2i[g1],node2i[g2],net_flow_edge[g1][g2]
                max_flow.AddArcWithCapacity(node2i[g1],node2i[g2],int(net_flow_edge[g1][g2]*edge_mult))
                #print node2i[g1],node2i[g2],int(net_flow_edge[g1][g2]*edge_mult)
                
        if max_flow.Solve(node2i[source_node],node2i['@super target']) == max_flow.OPTIMAL:
            fout = open(out_fname,'w')
            fout.write('source\ttarget\tweight\n')
            #print('Max flow:', max_flow.OptimalFlow())
            #print('')
            #print('  Arc    Flow / Capacity')
            nedge = 0
            for i in range(max_flow.NumArcs()):
                if max_flow.Flow(i)>0:
                    nedge += 1
                    fout.write(i2node[max_flow.Tail(i)]+'\t'+i2node[max_flow.Head(i)]+'\t'+str(max_flow.Flow(i))+'\n')
            fout.close()
        else:
            print('There was an issue with the max flow input.')
        if nedge>1:
            #print('Max flow:', max_flow.OptimalFlow())
            #print('')
            #print('  Arc    Flow / Capacity')
            
            
            for i in range(max_flow.NumArcs()):
                if max_flow.Flow(i)>0:
                    edge_list.append([i2node[max_flow.Tail(i)],i2node[max_flow.Head(i)],max_flow.Flow(i),max_flow.Capacity(i)])
                    node_list.add(i2node[max_flow.Tail(i)])
                    node_list.add(i2node[max_flow.Head(i)])
                    #print('%1s -> %1s   %3s  / %3s' % (i2node[max_flow.Tail(i)],i2node[max_flow.Head(i)],max_flow.Flow(i),max_flow.Capacity(i)))
                    pass
            break
    return edge_list,node_list

# read data

In [2]:
os.chdir('/srv/local/work/swang141/PatientSetAnnotation/Sheng/src/network_flow/')
file = '../../result/processed_word_net/TCGA/aggregate'
wct = {}
ect = {}
ect_ref = {}
ct = 0
fin = open(file)
for line in fin:
    w = line.strip().split('\t')
    w1,c1,w2,c2,c12 = w[0:5]
    c12 = int(c12)
    c1 = int(c1)
    c2 = int(c2)
    ct = ct + 1
    if c12<=3:
        continue
    if w1 not in ect:
        ect[w1] = {}
        ect_ref[w1] = {}
    if w2 not in ect:
        ect[w2] = {}
        ect_ref[w2] = {}
    if ct%30000000==0:
        print ct,ct*1.0/106424522,c12,w1,c1,w2,c2
    ect[w1][w2] = c12
    ect[w2][w1] = c12
    wct[w1] = c1#wct.get(w1,0) + c12
    wct[w2] = c2#wct.get(w2,0) + c12
    #ect_ref[w1][w2] = '\t'.join(w[5:])
    #ect_ref[w2][w1] = '\t'.join(w[5:])
fin.close()


30000000 0.281889920069 8 no-reflow 1955 coronary interventions 1266
60000000 0.563779840139 4 tubular networks 215 melanoma tumor cells 216
90000000 0.845669760208 4 podoplanin 2190 inflammatory cell infiltration 4777


In [56]:
from scipy import stats
generate_stop_word_list = False
stop_words = set()

if generate_stop_word_list:        
    gene_list = []
    for g in node_type:
        if node_type[g] == 'hpo_disease' and g in wct and wct[g]>10:
            gene_list.append(g)
    print len(gene_list)
    fout = open('../../data/dictionary/stopwords.txt','w')
    sort_x = sorted(wct.items(),key=operator.itemgetter(1))    
    sort_x.reverse()    
    new_d = {}
    for i in range(50000):
        w = sort_x[i][0]
        p_list = []
        q_list = []
        for g in gene_list:
            if g in ect[w]:
                p_list.append(ect[w][g]*1.0)
            else:
                p_list.append(0.)
            q_list.append(wct[g]*1.0)
        p_list = np.array(p_list) / np.sum(np.array(p_list))
        q_list = np.array(q_list) / np.sum(np.array(q_list))
        kl = stats.entropy(p_list,q_list)
        new_d[w] = kl
    sort_x = sorted(new_d.items(),key=operator.itemgetter(1))   
    for i in range(50000):
        w = sort_x[i][0]
        #print kl,np.sum(q_list),np.sum(p_list)
        if w not in node_type:
            node_type[w] = 'unknown'
        fout.write(w+'\t'+node_type[w]+'\t'+str(sort_x[i][1])+'\t'+str(wct[w])+'\n')
        stop_words.add(w)
    fout.close()
else:
    fin = open('../../data/dictionary/stopwords.txt')
    for line in fin:
        w = line.strip().split('\t')[0]
        sc = float(line.strip().split('\t')[2])
        if sc > 0.9:
            continue
        stop_words.add(w)
    fin.close()

In [57]:
print len(stop_words)

9205


In [9]:
import collections
node_type = {}
literome_g2g = collections.defaultdict(dict)
###process gene layer
fin = open('../../data/gene/pubmed.network')
for line in fin:
    g1,g2,type,conf = line.lower().strip().split('\t')
    literome_g2g[g1][g2] = 100
    literome_g2g[g2][g1] = 100
    node_type[g1] = 'gene'
    node_type[g2] = 'gene'
fin.close()

fin = open('../../data/gene/description.txt')
synonym_g2g = collections.defaultdict(dict)
description_t2g = collections.defaultdict(dict)
ct = 0
for line in fin:
    w = line.lower().strip().split('@')
    if len(w)>3:
        continue
    id,gl,text = w
    gl = gl.split(',')
    for g1 in gl:
        for g2 in gl:
            if g1==g2 or g1 not in wct or g2 not in wct:
                continue
            synonym_g2g[g1][g2] = 100
            synonym_g2g[g2][g1] = 100
            node_type[g1] = 'gene'
            node_type[g2] = 'gene'            
    text = line.lower().translate(None, ',:()=%>/[]')
    sent_l = text.strip().split('.')
    sent_ct = 0
    for sent in sent_l:
        pset = {}
        wl = sent.split(' ')
        w = ''
        for i in range(len(wl)):
            tmp_w = ''
            for k in range(5):
                if i+k >= len(wl):
                    break
                tmp_w += wl[i+k]
                if tmp_w in wct and tmp_w not in stop_words:
                    pset[tmp_w] = i                            
                tmp_w += ' '
        for ti in pset.keys():
            for tj in pset.keys():
                if ti in tj and ti!=tj:
                    pset.pop(ti, None)
        for w in pset:
            for g in gl:
                description_t2g[w][g] = 100 
                node_type[g] = 'gene'
                node_type[w] = 'ncbi_function'
                #print g,w
    ct += 1
    if ct%10000==0:
        print ct
fin.close()



10000
20000
30000
40000
50000


In [14]:
fin = open('../../data/dictionary/AutoPhrase_multi-words.txt.only_GO_based')
for line in fin:
    w = line.lower().strip().split('\t')
    sc = float(w[0])
    if sc<0.8:
        continue
    node_type[w[1]] = 'Han' + '_function'
fin.close()

fin = open('../../data/Relation/all_entity.txt')
for line in fin:
    w  =line.strip().lower().split('\t')
    if w[1] == 'disease':
        w[1] = 'disease'
    if w[0].replace('_',' ') in node_type:
        node_type[w[0].replace('_',' ')] ='Percha_'+w[1]
    else:
        node_type[w[0].replace('_',' ')] ='Percha_'+w[1]
fin.close()


fin = open('../../data/dictionary/hpo/20180118/phenotype_annotation.tab')
for line in fin:
    w = line.strip().lower().split('\t')
    node_type[w[2]] = 'hpo_disease'
fin.close()

fin = open('../../data/dictionary/hpo/20180118/hp_obo_parsed.tsv')
for line in fin:
    w = line.strip().lower().split('\t')
    w1 = w[0].split('(')[1].replace(')','')
    w2 = w[1].split('(')[1].replace(')','')
    node_type[w1] = 'hpo_disease'
    node_type[w2] = 'hpo_disease'
fin.close()

fin = open('../../data/disease/tissue.txt')
for line in fin:
    w =line.strip().lower()
    node_type[w] = 'tissue'
fin.close()

mesh_mapping = {'A':'tissue','B':'tissue','C':'disease','D':'drug','E':'drug','F':'disease',
               'G':'function','H':'entity','I':'entity','J':'entity','K':'entity','M':'entity','L':'entity',
               'N':'entity','V':'entity','Z':'entity'}
fin = open('../../data/dictionary/2017MeshTree.txt')
fin.readline()
for line in fin:
    w =line.strip().lower().split('\t')
    t = mesh_mapping[w[0][0].upper()]
    node_type[w[2]] = 'mesh' + '_' + t
fin.close()

fin = open('../../data/dictionary/go_term.txt')
for line in fin:
    w  =line.strip().lower()
    node_type[w] = 'GO_function'
fin.close()


gene_set_file = '../../data/drug/top_genes_exp_hgnc.txt'
fin = open(gene_set_file)
for line in fin:
    w = line.lower().strip().split('\t')
    node_type[w[0]] = 'gene' 


source_set = set()
use_disease = True

fin = open('../../data/disease/cancer_gene.txt')
result_dir = '../../result/network_flow/cancer/'
d2g = {}
background_gene = set()
for line in fin:
    w = line.lower().strip().split('\t')    
    if w[1] not in ect:
        continue
    if w[1] not in d2g:
        d2g[w[1]] = set()
    d2g[w[1]].add(w[0])
    node_type[w[0]] = 'gene'
    node_type[w[1]] = 'disease'
    background_gene.add(w[0])
fin.close()


print node_type['lung adenocarcinoma']


'''
if not use_disease:
    topk_tgt = 10
    result_dir = '../../result/network_flow/CTPR_drug/'
    d2dname = {}
    fin = open('../../data/drug/drug_map.txt')
    cancer2phrase = {}
    for line in fin:
        w = line.lower().strip().split('\t')    
        d2dname[w[2]] = w[0]
    fin.close()
    d2g = {}
    gene_set_file = '../../data/drug/top_genes_exp_hgnc.txt'
    fin = open(gene_set_file)
    for line in fin:
        w = line.lower().strip().split('\t')
        d = d2dname[w[1]]
        node_type[d] = 'drug'
        if w[0] not in ect:
            continue
        if d not in d2g:
            d2g[d] = {}
        d2g[d][w[0]] = abs(float(w[2]))
        node_type[w[0]] = 'gene'        
elif use_disease:
    topk_tgt = 1000
    result_dir = '../../result/network_flow/disease/'
    d2g = {}
    gene_set_file = '../../data/disease/gene_disease_formatted.txt'
    fin = open(gene_set_file)
    for line in fin:
        d,g = line.lower().strip().split('\t')
        node_type[d] = 'disease'
        if d not in d2g:
            d2g[d] = {}
        if g not in ect or d not in ect:
            continue
        d2g[d][g] = 1.
        node_type[g] = 'gene'
'''

disease


"\nif not use_disease:\n    topk_tgt = 10\n    result_dir = '../../result/network_flow/CTPR_drug/'\n    d2dname = {}\n    fin = open('../../data/drug/drug_map.txt')\n    cancer2phrase = {}\n    for line in fin:\n        w = line.lower().strip().split('\t')    \n        d2dname[w[2]] = w[0]\n    fin.close()\n    d2g = {}\n    gene_set_file = '../../data/drug/top_genes_exp_hgnc.txt'\n    fin = open(gene_set_file)\n    for line in fin:\n        w = line.lower().strip().split('\t')\n        d = d2dname[w[1]]\n        node_type[d] = 'drug'\n        if w[0] not in ect:\n            continue\n        if d not in d2g:\n            d2g[d] = {}\n        d2g[d][w[0]] = abs(float(w[2]))\n        node_type[w[0]] = 'gene'        \nelif use_disease:\n    topk_tgt = 1000\n    result_dir = '../../result/network_flow/disease/'\n    d2g = {}\n    gene_set_file = '../../data/disease/gene_disease_formatted.txt'\n    fin = open(gene_set_file)\n    for line in fin:\n        d,g = line.lower().strip().split('

In [1089]:
edge_type = []
edge_type.append(['disease','symptom'])
edge_type.append(['disease','tissue'])
edge_type.append(['disease','function'])
edge_type.append(['disease','disease'])
edge_type.append(['disease','gene'])

edge_type.append(['tissue','function'])
edge_type.append(['tissue','gene'])
edge_type.append(['tissue','symptom'])
edge_type.append(['tissue','tissue'])

edge_type.append(['symptom','function'])
edge_type.append(['symptom','tissue'])
edge_type.append(['symptom','symptom'])
edge_type.append(['symptom','gene'])

#edge_type.append(['function','gene'])
edge_type.append(['function','function'])

#edge_type.append(['gene','gene'])
print node_type['lung adenocarcinoma']

disease


In [58]:
kg_thres = 50000
kg_relation = collections.defaultdict(dict)
fin = open('../../data/Relation/all_relation.txt')

for line in fin:
    w  =line.strip().lower().split('\t')
    w1 = w[0]
    w2 = w[1]
    if w1 in stop_words or w2 in stop_words:
        continue
    if w1 not in node_type or w2 not in node_type:
        continue
    if len(w1) < 5 and (w1 not in node_type or node_type[w1]=='entity'):
        continue
    if len(w2) < 5 and (w2 not in node_type or node_type[w2]=='entity'):
        continue
    conf = float(w[3])
    conf = 100
    if conf < kg_thres:
        continue
    sent = w[4]
    if [node_type[w1],node_type[w2]] in edge_type:
        kg_relation[w1][w2] = (sent,conf)
        continue
    if [node_type[w2],node_type[w1]] in edge_type:
        kg_relation[w2][w1] = (sent,conf)
        continue
fin.close()


gene_ontology = collections.defaultdict(dict)

fin = open('../../data/Relation/GO_term.network')
for line in fin:
    w  =line.strip().lower().split('\t')
    w1 = w[0]
    w2 = w[1]
    gene_ontology[w1][w2] = 100
fin.close()

print node_type['lung adenocarcinoma']

disease


In [59]:
from scipy import stats
net = collections.defaultdict(dict)
bp_net = collections.defaultdict(dict)
topk = 5000
sent_ct = 239833652
ct = 0
max_ngh = 100
for k1 in ect:
    ct += 1
    if k1 in stop_words:
        continue
    new_d = {}
    for k2 in ect[k1]:
        if k2 in stop_words:
            continue
        #if k2 not in node_type or k1 not in node_type:
        #    continue
        #if False:#[node_type[k1],node_type[k2]] not in edge_type:
        #    continue
        #if node_type[k1] == node_type[k2] and wct[k1] < wct[k2]:
            #new_d[k2] = -1            
        #else:
        value = ect[k1][k2]*1.0 / wct[k2] / wct[k1] *239833652  
        if value <= 20:
            continue
        #table = np.zeros((2,2))        
        #table[0,1] = wct[k2] - ect[k1][k2]
        #table[1,0] = wct[k1] - ect[k1][k2]
        #table[1,1] = ect[k1][k2]
        #table[0,0] = sent_ct - table[0,1] - table[1,1] - table[1,0]
        #oddsratio, pvalue = stats.fisher_exact([[8, 2], [1, 5]])
        #new_d[k2] = - np.log10(pvalue)
        #if ect[k1][k2]*1.0 / wct[k2]<0.01:
        #    continue
        new_d[k2] = value
    sort_x = sorted(new_d.items(),key=operator.itemgetter(1))   
    sort_x.reverse()
    for i in range(min(len(sort_x),max_ngh)):
        k2 = sort_x[i][0]
        sc = sort_x[i][1]            
        net[k1][k2] = sc
        bp_net[k2][k1] =sc
    if ct%100000==0:
        print 'finished',ct*1.0/len(ect)

'''
for g in kg_relation:
    for t in kg_relation[g]:        
        bp_net[t][g] = kg_relation[g][t][1]
        net[g][t] = kg_relation[g][t][1]
   
database_relation_l = [gene_ontology,description_t2g,literome_g2g]

for dr in database_relation_l:
    for g in dr:
        for t in dr[g]:
            net[g][t] = dr[g][t]
            bp_net[t][g] = dr[g][t]
'''         

finished 0.133534168723
finished 0.267068337446
finished 0.400602506169
finished 0.534136674892
finished 0.667670843615
finished 0.801205012339
finished 0.934739181062


'\nfor g in kg_relation:\n    for t in kg_relation[g]:        \n        bp_net[t][g] = kg_relation[g][t][1]\n        net[g][t] = kg_relation[g][t][1]\n   \ndatabase_relation_l = [gene_ontology,description_t2g,literome_g2g]\n\nfor dr in database_relation_l:\n    for g in dr:\n        for t in dr[g]:\n            net[g][t] = dr[g][t]\n            bp_net[t][g] = dr[g][t]\n'

In [ ]:
node_type['c3orf21']

In [1309]:
#k1 = 'lung cancer'
k1  = 'mutant egfr'
k2 = 'lung adenocarcinoma'
table = np.zeros((2,2))        
table[0,1] = wct[k2] - ect[k1][k2]
table[1,0] = wct[k1] - ect[k1][k2]
table[1,1] = ect[k1][k2]
table[0,0] = sent_ct - table[0,1] - table[1,1] - table[1,0]
oddsratio, pvalue = stats.fisher_exact(table)
print pvalue
print ect[k1][k2]*1.0 / wct[k2] / wct[k1] *239833652  
print table

KeyboardInterrupt: 

In [1346]:
#rank genes
max_ngh = 10
for max_path in [4]:
    for edge_thres in [100]:
        dis2source = {}
        dis2target = {}
        if not os.path.exists(result_dir):
            os.makedirs(result_dir)
        dct = 0
        gene_score = {}
        for d in d2g:
            dct = dct + 1
            if d!='lung adenocarcinoma':
                continue
            print d
            if d not in net:
                continue     
            gene_score[d] = {}
            gct = 0.
            for g in background_gene:
                gct += 1
                #if g!='zmym2':
                #    continue
                if g in bp_net and len(bp_net[g])>0:
                    target_set = set()
                    target_set.add(g)
                    source_node = d
                    net_flow_edge,node2i,i2node,select_node_set = extract_sub_graph(net,bp_net,source_node,target_set,max_path,edge_thres,max_ngh)  
                    edge_weight = -1
                    nedge = 0
                    if source_node not in select_node_set:
                        #print source_node,'not found',g,len(select_node_set)
                        pass
                    else:
                        print 'nnode',len(select_node_set)
                        out_fname = result_dir + d+'_'+g
                        #edge_list, node_list,net_edge_weight = dfs_with_regard_source_target(source_node,g,max_path)
                        edge_list, node_list = dfs_find_graph(edge_type,source_node,'@super target',
                                                              select_node_set,net_flow_edge,node2i,i2node,node_type,max_path+1)
                        #edge_list, node_list = run_network_flow(out_fname,source_node,select_node_set,net_flow_edge,node2i,i2node)
                        nedge = len(edge_list) 
                        type_d = {}
                        for e in edge_list:        
                            e1,e2,w,mw = e
                            if e1 in node_type and e2 in node_type:
                                type_d[(node_type[e1],node_type[e2])] = type_d.get((node_type[e1],node_type[e2]),0) + 1
                            #print e1,e2,w,mw
                            if e1=='@super target' or e2=='@super target':
                                edge_weight = float(w)
                            #if e2==g:
                            #    print e1,e2,w,mw,select_edge_ct[e1][g]
                            #if e1==d:
                            #    print e1,e2,w,mw,select_edge_ct[e1][e2]
                        #sort_x = sorted(type_d.items(),key=operator.itemgetter(1))   
                        #sort_x.reverse()
                        #for i in range(len(sort_x)):
                        #    print sort_x[i][0],sort_x[i][1]
                        
                        plot_figure(node_list,edge_list,result_dir + d+'_'+g,net_flow_edge)
                        
                    if True:#edge_weight < 10000:# and edge_weight!=-1:
                        gene_score[d][g] = edge_weight
                        print max_path,max_edge,g,d,edge_weight,g in d2g[d],get_auc_per_drug(gene_score,d2g,d),nedge
            print max_path,max_edge,d,get_auc_per_drug(gene_score,d2g,d)

            #IFrame("test.pdf", width=600, height=300)


lung adenocarcinoma
nnode 1283
4 10 zmym2 lung adenocarcinoma -1 True nan 0
nnode 2571
4 10 plcb1 lung adenocarcinoma 328508.0 False 0.0 13
nnode 7006
4 10 alk lung adenocarcinoma 61989.0 False 0.0 7
nnode 1528
4 10 syncrip lung adenocarcinoma -1 True 0.0 0
nnode 1212
4 10 cct5 lung adenocarcinoma 35986.0 True 0.0 5
nnode 3776
4 10 trip10 lung adenocarcinoma -1 False 0.22222222222222224 0
nnode 9292
4 10 cdkn2a lung adenocarcinoma 127057.0 True 0.3333333333333333 5
nnode 5763
4 10 syk lung adenocarcinoma -1 False 0.4375 0
nnode 1916
4 10 hcfc1 lung adenocarcinoma -1 False 0.49999999999999994 0
nnode 9073
4 10 rac1 lung adenocarcinoma 49767.0 False 0.45833333333333337 5
nnode 2408
4 10 rfc4 lung adenocarcinoma -1 False 0.5 0
nnode 1633
4 10 magi2 lung adenocarcinoma -1 False 0.53125 0
nnode 3474
4 10 hsp90aa1 lung adenocarcinoma -1 True 0.4875 0
nnode 2633
4 10 tfdp2 lung adenocarcinoma -1 False 0.5111111111111111 0
nnode 2746
4 10 tfdp1 lung adenocarcinoma 18355.0 True 0.53703703703703

KeyboardInterrupt: 

In [1347]:
net['breast']

30941

In [1359]:
ct = 0
sent_ct = 239833652
w = 'breast invasive carcinoma'
new_d = {}
for w1 in net[w]:
    new_d[w1] = ect[w][w1] * 1.0/ wct[w1] / wct[w] * sent_ct
sort_x = sorted(new_d.items(),key=operator.itemgetter(1))   
sort_x.reverse()
for i in range(min(len(sort_x),50)):
    print sort_x[i][0],wct[sort_x[i][0]],sort_x[i][1],ect[w][sort_x[i][0]] * 1.0/ wct[sort_x[i][0]],ect[w][sort_x[i][0]] * 1.0/ wct[w]

sema6d 75 290707.45697 0.0533333333333 0.0909090909091
tcga 2210 64126.6449198 0.0117647058824 0.590909090909
cancer genome atlas 2089 52185.3979721 0.00957395883198 0.454545454545
brca 1851 47116.2815186 0.00864397622907 0.363636363636
lung squamous cell carcinoma 849 25680.870757 0.00471142520612 0.0909090909091
primary chemotherapy 1781 12242.0321576 0.00224592925323 0.0909090909091
carcinoma cases 1912 11403.2736782 0.00209205020921 0.0909090909091
regulatory subunits 2236 9750.92096276 0.00178890876565 0.0909090909091
glioblastoma multiforme 9340 5835.93663617 0.00107066381156 0.227272727273
lung adenocarcinoma 6502 3353.28503118 0.000615195324516 0.0909090909091
c-erbb-2 7533 2894.339476 0.000530996946768 0.0909090909091
breast cancer cell lines 10035 2172.70147212 0.00039860488291 0.0909090909091
gbm 20230 1616.63810722 0.000296589223925 0.136363636364
dataset 37829 1440.89582547 0.000264347458299 0.227272727273
patient survival 24811 878.765840665 0.000161218814236 0.0909090909

In [1353]:
bp_net[w]

{}

In [1270]:
len(select_node_set)

3795

In [987]:
stop_words.add('snps')
stop_words.add('snp')

def combine_str(str1,str2):
    s1 = str1.split('#')
    s2 = str2.split('#')
    s = ''
    for i in range(0,len(s1)-1):
        s += s1[i]+'#'
    s += str2
    return s

def dfs_find_strong_evidence_graph(source_node,target_node,ect,depth,used_node):
    if depth==0:
        return [source_node + '#' + target_node]
    all_path = set()
    sc = {}
    for w in wct:
        if source_node not in node_type or w not in node_type or target_node not in node_type:
            continue
        if [node_type[source_node],node_type[w]] not in edge_type:
            continue
        if [node_type[w],node_type[target_node]] not in edge_type:
            continue
        if node_type[w] == node_type[target_node] and node_type[w]!='gene' and wct[w] < wct[target_node]:
            continue
        if node_type[source_node] == node_type[w] and node_type[w]!='gene' and wct[source_node] < wct[w]:
            continue            
        if w in ect[source_node] and w in ect[target_node] and w not in stop_words and w not in used_node:
            sc[w] = ect[source_node][w]*ect[target_node][w]*1.0/wct[target_node]/wct[w]/wct[source_node]/wct[w]
    sort_x = sorted(sc.items(),key=operator.itemgetter(1))   
    sort_x.reverse()
    if len(sort_x)==0:
        return ""
    w = sort_x[0][0]
    next_used_node = set()
    for s in used_node:
        next_used_node.add(s)
    next_used_node.add(w)
    path_tgt = dfs_find_strong_evidence_graph(w,target_node,ect,depth-1,next_used_node)
    path_src = dfs_find_strong_evidence_graph(source_node,w,ect,depth-1,next_used_node)
    #print w,path,source_node,target_node
    '''
    if type(path_tgt)==str:
        all_path.append(source_node + '#' + path)
    else:
        for p in path:
            all_path.append(source_node + '#' + p)
    '''
    #print 'tgt',depth,path_tgt,w,target_node
    #print 'src',depth,path_src,source_node,w
    for pt in path_tgt:
        for ps in path_src:
            all_path.add(combine_str(ps,pt))
            #print 'cb',depth,combine_str(ps,pt)
    for pt in path_tgt:
        all_path.add(source_node+'#'+pt)
        #print 'cb',depth,source_node+'#'+pt
    for ps in path_src:
        all_path.add(ps+'#'+target_node)
        #print 'cb',depth,ps+'#'+target_node 
    if target_node in ect[source_node]:
        all_path.add(source_node+'#'+target_node)
    return all_path

def dfs_with_regard_source_target(source_node,target_node,path_depth=3):
    #source_node = 'lung adenocarcinoma'
    #target_node = 'hsp90aa1'
    used_node = set()
    all_path = dfs_find_strong_evidence_graph(source_node,target_node,ect,path_depth,used_node)
    extracted_net_flow_edge = {}
    extracted_select_node_set = set()
    extracted_net_flow_edge[target_node] = {}
    for path in all_path:
        w = path.split('#')
        #print path
        for i in range(0,len(w)-1):
            w1 = w[i]
            w2 = w[i+1]
            if w1 not in extracted_net_flow_edge:
                extracted_net_flow_edge[w1] = {}
            extracted_net_flow_edge[w1][w2] = ect[w1][w2]*1000000.0/(wct[w1]+1)/(wct[w2]+1)
            extracted_select_node_set.add(w1)
            extracted_select_node_set.add(w2)        

    extracted_select_node_set.add('@super target')    
    extracted_net_flow_edge['@super target'] = {}
    extracted_net_flow_edge[target_node]['@super target'] = 1000000
    extracted_net_flow_edge['@super target'][target_node] = 1000000

    node2i = {}
    i2node = {}
    for s in extracted_select_node_set:
        nnode = len(node2i)
        i2node[nnode] = s
        node2i[s] = nnode
    if target_node not in node2i:
        return [],set(),{}
    edge_list, node_list = run_network_flow('tmp',source_node,extracted_select_node_set,extracted_net_flow_edge,node2i,i2node)
    return edge_list, node_list, extracted_net_flow_edge


In [996]:
w1 ='lung adenocarcinoma'
w2 = 'tfdp2'
sc = {}
for w in wct:
    if w in ect[w1] and w in ect[w2] and w not in stop_words:
        #print w,ect[w1][w]*ect[w2][w]*1.0/wct[w2]/wct[w]/wct[w1]
        sc[w] = ect[w1][w]*ect[w2][w]*1.0*239833652*239833652/wct[w2]/wct[w]/wct[w1]/wct[w]
sort_x = sorted(sc.items(),key=operator.itemgetter(1))   
sort_x.reverse()
print sort_x

[('dach1', 6313.713174741137), ('e2f1', 50.457841580560526), ('cell cycle', 0.33949684195946794), ('transcription factor', 0.20332001223212323)]


In [1026]:
topk

3

In [ ]:
d = 'thyroid'
g = 'plcb1'
print result_dir + d+'_'+g+".pdf"
IFrame(result_dir + d+'_'+g+".pdf", width=1200, height=600)

KeyboardInterrupt: 

In [657]:
if not os.path.exists(result_dir):
    os.makedirs(result_dir)
dct = 0
for d in d2g:
    dct = dct + 1
    #if d!='asthma':
    #    continue
    if d not in select_edge_ct:
        continue
    target_set = set()
    #sort_x = sorted(d2g[d].items(),key=operator.itemgetter(1)) 
    #sort_x.reverse()
    #for i in range(min(len(sort_x),topk_tgt)):
    #    target_set.add(sort_x[i][0])
    for g in background_gene:
        if g in select_edge_ct:
            target_set.add(g)
    #target_set = background_gene
    source_node = d
    net_flow_edge,node2i,i2node,select_node_set = extract_sub_graph(select_edge_ct,rev_select_edge_ct,
                                                                    source_node,target_set,3)  
    if source_node not in select_node_set:
        continue
    out_fname = result_dir + d
    edge_list, node_list = run_network_flow(out_fname,source_node,select_node_set,net_flow_edge,node2i,i2node)
    fout = open(out_fname + '_sentence.txt','w')
    fout_backgroud  = open(out_fname + '_sentence_candidates.txt','w')
    nedge = len(edge_list)
    print d,nedge
    if nedge == 0:
        continue
    dot = Digraph(comment='Network flow')
    for node in node_list:
        if node==source_node:
            dot.attr('node',color='blue')
            dot.node(node)
        elif node in d2g[d]:
            dot.attr('node',color='red')
            dot.node(node)            
        dot.attr('node',color='black')
        dot.node(node)                
    for e in edge_list:        
        e1,e2,w,mw = e
        if e1 in kg_relation and e2 in kg_relation[e1]:
            print 'find'
            label = 'Percha et al.'
        else:
            label = 'text mining'
        if e1 not in node_type:
            t1 = 'entity'
        else:
            t1 = node_type[e1]
        if e2 not in node_type:
            t2 = 'entity'
        else:
            t2 = node_type[e2]
        dot.edge(e1,e2,label=label+'('+t1+'-'+t2+':'+str(w)+'))
        if e1!='@super target' and e2!='@super target':
            #sent = extract_sent(ect_ref,e1,e2)
            sent,path,sent_sc,path_l = find_sent(e2,e1)
            print e1,e2,sent,path,sent_sc
            fout.write(e1+'->'+e2+'\t'+str(sent_sc)+'\n'+path+'\n'+sent+'\n')
            fout_backgroud.write(e1+'->'+e2+'\t'+str(sent_sc)+'\n'+path+'\n'+sent+'\n')
            for p in path_l:
                fout_backgroud.write(p+'\n')
            fout.write('\n')
            fout_backgroud.write('\n')
    dot.render(result_dir + d+'_fig') 
    fout.close()
    fout_backgroud.close()
    #IFrame("test.pdf", width=600, height=300)
    #break

KeyboardInterrupt: 

In [995]:
sent_ct = 0
for pid in range(400):
    fin = open('../../data/pubmed/pmid2meta_autophrase.chunk'+str(pid))
    ct = 0
    for line in fin:
        ct = ct + 1
        sl = line.strip().split('.')
        sent_ct += len(sl)
    fin.close()
    print pid
print sent_ct

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [60]:
a={[}'s','b','e'}
for i,j in enumerate(a):
    print i,j

0 s
1 b
2 e


# New heading